In [1]:
# pip install bitsandbytes accelerate flash-attn

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [3]:
import torch
import bz2
import pickle
import importlib
from tqdm import tqdm


import time
from IPython.display import JSON
import sys
sys.settrace(None)
import pdb

import copy

import psycopg
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TextStreamer, StoppingCriteria, StoppingCriteriaList, DynamicCache, OffloadedCache #, CodeGenTokenizer
from transformers.generation.logits_process import LogitsProcessorList

In [4]:
assert torch.cuda.is_available()

In [7]:
# index_module = importlib.import_module('phi4_index')
index_module = importlib.import_module('qwen25_index')
Index = getattr(index_module, 'Index')
index = Index()

# model_module = importlib.import_module('phi4_model')
model_module = importlib.import_module('qwen25_1B_model')
Model = getattr(model_module, 'Model')
model = Model()

Loading Qwen/Qwen2.5-1.5B-Instruct


In [8]:
streamer = TextStreamer(model.tokenizer)

In [9]:
from ctrie import ConstrainedLogitsProcessor, ConstrainedStateList, \
                    ConstrainedState, DictIndex, GetAnswer

In [10]:
question = {'role':'user', 'content': '''Which city is the capital of the country where the Eiffel Tower is?
'''}
question = {'role':'user', 'content': '''Who was the quarterback of the team that won Super Bowl 50?
'''}
question = {'role':'user', 'content': '''Who is the lead character in the Uncharted series?
'''}
question = {'role':'user', 'content': '''Who played Gandalf in the Lord of the Rings movies?
'''}
question = {'role':'user', 'content': '''What was Nirvana's last studio album called?
'''} # TODO test with beamsearch 3. with beam search if I have only 1 allowed token the other beams could generate something illegal and than crash
# TODO understand the crash is due to beam search and abandon those illegal paths. or try to explore always different triples with beam search???
"""
question = {'role':'user', 'content': '''Is Johnny Depp or Brad Pitt older?
'''}
question = {'role':'user', 'content': '''How many members were there in the original NSYNC music group?
'''}
"""

question = 'Who influenced Nirvana band?'

prompted_texts = [model.apply_prompt_template(question)]

In [11]:
print(prompted_texts[0])

<|im_start|>system
You are a helpful question answering assistant that bases its answer on facts from a knowledge base.
1) You receive an input question.
2) You reason on the path you need to follow to reach the answer starting from the information in the question.
3) You explicitly provide relevant facts, one per line starting with "Fact:".
4) You explain your reasoning process and provide a long answer with your motivations based on the facts.
5) You provide a short concise answer.

Sometimes it could happen that there are no relevant facts in the knowledge base.
In these cases you either:
a) provide your own answer clearly stating that there were no supporting facts to your answer;
b) in case you don't know the answer or you are not confident enough, you just answer "I don't know".
<|im_end|>
<|im_start|>user
Which mountain is taller between Mont Blanc and Mount Rainier?
<|im_end|>
<|im_start|>assistant
Reasoning: I need to provide the height of Mont Blanc and the height of Mount Ra

In [12]:
inputs = model.tokenizer(prompted_texts, return_tensors='pt', padding=True, padding_side='right')
inputs = inputs.to(model.device)

In [13]:
getanswer = GetAnswer(model.answer_tokens, model.eofanswer, all)
stopping_criteria = StoppingCriteriaList([
    getanswer
])

In [14]:
torch.cuda.empty_cache()

In [15]:
num_beams = 1

auto_streamer = streamer if num_beams == 1 else None

states = ConstrainedStateList(
    [ConstrainedState(
                begin_pattern = model.switch_pattern,
                end_pattern = model.newline_token,
                cache_index = DictIndex(end_of_triple=index.end_of_triple),
                subtree_cache = DictIndex(end_of_triple=index.end_of_triple),
                oneleaf_cache = DictIndex(end_of_triple=index.end_of_triple)
            ) for _ in range(num_beams)])

constrained_processor = ConstrainedLogitsProcessor(
    index=index.index,
    end_token=model.newline_token, states=states)#, tokenizer=tokenizer)
logits_processor_list = LogitsProcessorList([
    constrained_processor
])

model.model.eval()
start = time.time()

with torch.no_grad():

    getanswer.set_prompt(inputs.input_ids[0])

    #pdswitch_patternb.set_trace()
    out = model.model.generate(
        **inputs,
        logits_processor=logits_processor_list,
        max_new_tokens=200,
        streamer = auto_streamer,
        #do_sample = True,
        #top_k=3,
        num_beams=num_beams,
        #num_return_sequences=1,
        num_return_sequences=num_beams,
        #no_repeat_ngram_size=1,
        #remove_invalid_values=True,
        stopping_criteria=stopping_criteria,
        use_cache=True,
        #past_key_values=past_key_values,
        kwargs = {'constrained_state': states}, # passing state
    )

print('Elapsed', time.time() - start)

<|im_start|>system
You are a helpful question answering assistant that bases its answer on facts from a knowledge base.
1) You receive an input question.
2) You reason on the path you need to follow to reach the answer starting from the information in the question.
3) You explicitly provide relevant facts, one per line starting with "Fact:".
4) You explain your reasoning process and provide a long answer with your motivations based on the facts.
5) You provide a short concise answer.

Sometimes it could happen that there are no relevant facts in the knowledge base.
In these cases you either:
a) provide your own answer clearly stating that there were no supporting facts to your answer;
b) in case you don't know the answer or you are not confident enough, you just answer "I don't know".
<|im_end|>
<|im_start|>user
Which mountain is taller between Mont Blanc and Mount Rainier?
<|im_end|>
<|im_start|>assistant
Reasoning: I need to provide the height of Mont Blanc and the height of Mount Ra

In [37]:
torch.cuda.empty_cache()

In [44]:
_from = 0# len(inputs.input_ids[0])
for i in range(out.shape[0]):
    print('-'*30, sum(out[i][_from:]), len(out[i][_from:]))
    print(model.tokenizer.decode(out[i][_from:]))

------------------------------ tensor(5872727, device='cuda:0') 531
<|im_start|>system<|im_sep|>You are a helpful question answering assistant that bases its answer on facts from a knowledge base.
1) You receive an input question.
2) You reason on the path you need to follow to reach the answer starting from the information in the question.
3) You explicitly provide relevant facts, one per line starting with "Fact:".
4) You explain your reasoning process and provide a long answer with your motivations based on the facts.
5) You provide a short concise answer.

Sometimes it could happen that there are no relevant facts in the knowledge base.
In these cases you either:
a) provide your own answer clearly stating that there were no supporting facts to your answer;
b) in case you don't know the answer or you are not confident enough, you just answer "I don't know".<|im_end|><|im_start|>user<|im_sep|>Which mountain is taller between Mont Blanc and Mount Rainier?<|im_end|><|im_start|>assistan

In [42]:
for i, triple in enumerate(states[0].generated_triples):
    print(i, model.tokenizer.decode(triple)[:-1], triple, end='\n')

0  <Nirvana> <inception> <1832-01-01T00:00:00Z> . [366, 45, 404, 68559, 29, 366, 258, 1010, 29, 366, 10750, 17, 12, 1721, 12, 1721, 51, 410, 25, 410, 25, 410, 57, 29, 662, 198]
1  <Nirvana> <genre> <Folk music> . [366, 45, 404, 68559, 29, 366, 34713, 29, 366, 37, 23427, 4731, 29, 662, 198]
2  <Nirvana> <genre> <Contemporary folk music> . [366, 45, 404, 68559, 29, 366, 34713, 29, 366, 825, 14084, 29036, 4731, 29, 662, 198]
3  <Nirvana> <genre> <New-age music> . [366, 45, 404, 68559, 29, 366, 34713, 29, 366, 3648, 44041, 4731, 29, 662, 198]
4  <Nirvana> <genre> <J-pop> . [366, 45, 404, 68559, 29, 366, 34713, 29, 366, 41, 41352, 29, 662, 198]
5  <Nirvana> <genre> <Electropop> . [366, 45, 404, 68559, 29, 366, 34713, 29, 366, 30431, 897, 454, 29, 662, 198]
6  <Nirvana> <genre> <Intelligent dance music> . [366, 45, 404, 68559, 29, 366, 34713, 29, 366, 1090, 21149, 15612, 4731, 29, 662, 198]


In [52]:
stop, ans = getanswer.get_answer(out[0])
# TODO: is getanswer still needed?
stop, model.tokenizer.decode(list(ans)) # TODO trim and remove special tokens

(False, '')